In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLP_ACM_StructuredBenchmark

benchmark = DBLP_ACM_StructuredBenchmark(data_dir_path="../data/")
benchmark

18:30:36 INFO:Extracting DBLP-ACM-Structured...
18:30:36 INFO:Reading DBLP-ACM-Structured record_dict...
18:30:36 INFO:Reading DBLP-ACM-Structured train.csv...
18:30:36 INFO:Reading DBLP-ACM-Structured valid.csv...
18:30:36 INFO:Reading DBLP-ACM-Structured test.csv...


<DBLP_ACM_StructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-ACM/dblp_acm_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'authors', 'venue', 'year']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

In [9]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [10]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

18:30:41 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
18:30:41 INFO:Did not find folder stsb-distilbert-base
18:30:41 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
18:30:41 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
18:30:42 INFO:Use pytorch device: cuda


In [11]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

18:30:42 INFO:GPU available: True, used: True
18:30:42 INFO:TPU available: False, using: 0 TPU cores
18:30:42 INFO:Train positive pair count: 1332
18:30:42 INFO:Train negative pair count: 6085
18:30:42 INFO:Valid positive pair count: 444
18:30:42 INFO:Valid positive pair count: 2029
18:30:42 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
18:30:45 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 66.4 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.455   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

19:24:05 INFO:Loading the best validation model from ../tb_logs/matcher-DBLP-ACM-Structured/version_0/checkpoints/epoch=15-step=3711.ckpt...
19:24:05 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
19:24:05 INFO:Did not find folder stsb-distilbert-base
19:24:05 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
19:24:05 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
19:24:06 INFO:Use pytorch device: cuda


In [12]:
model.validate(datamodule)

{'valid_f1_at_0.3': tensor(0.9933),
 'valid_f1_at_0.5': tensor(0.9933),
 'valid_f1_at_0.7': tensor(0.9910),
 'valid_f1_at_0.9': tensor(0.9921),
 'valid_precision_at_0.3': tensor(0.9910),
 'valid_precision_at_0.5': tensor(0.9910),
 'valid_precision_at_0.7': tensor(0.9910),
 'valid_precision_at_0.9': tensor(0.9955),
 'valid_recall_at_0.3': tensor(0.9955),
 'valid_recall_at_0.5': tensor(0.9955),
 'valid_recall_at_0.7': tensor(0.9910),
 'valid_recall_at_0.9': tensor(0.9887)}

## Testing

In [13]:
model.test(datamodule)

19:25:48 INFO:Test positive pair count: 444
19:25:48 INFO:Test positive pair count: 2029


{'test_f1_at_0.3': tensor(0.9910),
 'test_f1_at_0.5': tensor(0.9910),
 'test_f1_at_0.7': tensor(0.9899),
 'test_f1_at_0.9': tensor(0.9898),
 'test_precision_at_0.3': tensor(0.9866),
 'test_precision_at_0.5': tensor(0.9888),
 'test_precision_at_0.7': tensor(0.9910),
 'test_precision_at_0.9': tensor(0.9932),
 'test_recall_at_0.3': tensor(0.9955),
 'test_recall_at_0.5': tensor(0.9932),
 'test_recall_at_0.7': tensor(0.9887),
 'test_recall_at_0.9': tensor(0.9865)}